In [2]:
import aiosqlite
from pathlib import Path
DB_VERSION = 2
DB_PATH = Path(f'/Users/karlkim/.chia/simulator/hints/db/blockchain_v2_simulator0.sqlite')

In [3]:
from chia.consensus.block_record import BlockRecord
query = f"""SELECT block_record FROM full_blocks"""
async with aiosqlite.connect(DB_PATH) as db:
    async with db.execute(query) as cursor:
        async for row in cursor:
            block_record = BlockRecord.from_bytes(row[0])
            print(block_record.header_hash.hex())

586d7cc57c40ab6dd4daafdc5de0008c30777ef5fb95102bcbb4e841373210d9

7d91b636f8174f505c6c81052b9a08647628244d6976652e61c10502db4d374c

b5aadeddb982b9815a2a0282b82aa2cded04cfef5a5c91b7b936c77fe68af7fe

24ca69f2ce331f1fd33d0353d2ead52c3655f69063724a728274925b26fe2d28

ec02ba90cbefa3b58863d2fadd1266d76e9e7de9250efdbd0f76ff1a632cf528

...

ac0fba04ec1651b20321447d931ab9d8180e74556a96c710c166ab1dc96e5c80

8d8b03c7d3546d546aa5a85ede1713e4227ce8a3f84c052b4fd8253dd759bdf6

5699f9de61bc18276749194d338d86462e2cf19e77bb5bd7ce526dc825016ec5

dacb884c94f5e1ad5d810a7b4f17c72f4be77575e04757b145e9f39e4d40c02d

cc5ac0b5e5c429ff2a09b0c21ad55f20242ad184f038854b50c234e7f289a032

In [4]:
# decorator
def with_db_connection(f):
    async def with_connection(*args, **kwargs):
        db = await aiosqlite.connect(DB_PATH)
        try:
            rv = await f(db, *args, **kwargs)
        except Exception:
            await db.rollback()
            raise
        else:
            await db.commit()  # or maybe not
        finally:
            await db.close()

        return rv

    return with_connection

In [5]:
@with_db_connection
async def get_height_async(db) -> Optional[uint32]:
    query_peak = "SELECT hash FROM current_peak WHERE key = 0"
    async with db.execute(query_peak) as cursor:
        peak_row = await cursor.fetchone()
        if peak_row is None:
            return None

    query_height = "SELECT height FROM full_blocks WHERE header_hash=?"
    async with db.execute(query_height, (peak_row[0],)) as cursor:
        height = await cursor.fetchone()
        if height is not None:
            return uint32(height[0])
        else:
            return None
await get_height_async()

11

In [6]:
import zstd

@with_db_connection
async def get_block_record_by_height_async(
    db, height: uint32
) -> Optional[BlockRecord]:
    query = "SELECT block_record FROM full_blocks WHERE height=?"
    async with db.execute(query, (height,)) as cursor:
        row = await cursor.fetchone()
        if row is not None:
            block_record = BlockRecord.from_bytes(row[0])
            return block_record
        else:
            return None

@with_db_connection
async def get_block_bytes_by_height_async(
    db, height: uint32
) -> Optional[str]:
    query = f"SELECT block FROM full_blocks WHERE height=?"
    async with db.execute(query, (height,)) as cursor:
        row = await cursor.fetchone()
        if row is not None:
            block_bytes = zstd.decompress(row[0])
            return block_bytes
        else:
            return None

async def get_block_by_height_async(height: uint32) -> Optional[FullBlock]:
    block_bytes = await get_block_bytes_by_height_async(height)
    if block_bytes is None:
        return None
    block = FullBlock.from_bytes(block_bytes)
    return block

def get_coin_record(row) -> CoinRecord:
    return CoinRecord(
        Coin(row[4], row[3], uint64.from_bytes(row[5])), row[0], row[1], row[2], row[6]
    )

@with_db_connection
async def get_coin_record_by_name_async(db, name: str):
    query = f"""SELECT confirmed_index, spent_index, coinbase, puzzle_hash, 
    coin_parent, amount, timestamp FROM coin_record WHERE coin_name=?"""

    async with db.execute(query, (bytes32.from_hexstr(name),)) as cursor:
        row = await cursor.fetchone()
        if row is None:
            return None
        else:
            return get_coin_record(row)

In [7]:
await get_block_record_by_height_async(2)

BlockRecord(
    header_hash=<bytes32: b5aadeddb982b9815a2a0282b82aa2cded04cfef5a5c91b7b936c77fe68af7fe>,
    prev_hash=<bytes32: 7d91b636f8174f505c6c81052b9a08647628244d6976652e61c10502db4d374c>,
    height=2,
    weight=3072,
    total_iters=20605,
    signage_point_index=14,
    challenge_vdf_output=ClassgroupElement(
        data=<bytes100: 00001d00010100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>
    ),
    infused_challenge_vdf_output=ClassgroupElement(
        data=<bytes100: 03000c00010301000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>
    ),
    reward_infusion_new_challenge=<bytes32: 756e903b6b6ce01db2aa31b6d2fe554a660dfbc6029618af47a84ce04edf4de7>,
    challenge_block_info_hash=<bytes32: e58141bbd3a5e656874a95d21ab609db7593b34f73f2d43cc5fc567492a3d308>,
    sub_slot_iters=1024,
    pool_puzzle_hash=<bytes32: 121cbc81114741b39f7cf6aa6180303a2f5fee9d0e5ef3986a0215ca46b41385>,
    farmer_puzzle_hash=<bytes32: 121cbc81114741b39f7cf6aa6180303a2f5fee9d0e5ef3986a0215ca46b41385>,
    required_iters=61,
    deficit=9,
    overflow=True,
    prev_transaction_block_height=1,
    timestamp=1668526174,
    prev_transaction_block_hash=<bytes32: 7d91b636f8174f505c6c81052b9a08647628244d6976652e61c10502db4d374c>,
    fees=0,
    reward_claims_incorporated=[
        Coin { parent_coin_info: eb8c4d20b322be8d9fddbf9412016bdf00000000000000000000000000000001, puzzle_hash: bd85c2c28bd770997c36037fa842fbbb719803dba7ff6b741148739576e68a39, amount: 1750000000000 },
        Coin { parent_coin_info: fe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000001, puzzle_hash: bd85c2c28bd770997c36037fa842fbbb719803dba7ff6b741148739576e68a39, amount: 250000000000 }
    ],
    finished_challenge_slot_hashes=[
        <bytes32: 83616d03da35d176e4c757f317e3c86a320adc3d495354e7c1330cfccc8cc8a9>
    ],
    finished_infused_challenge_slot_hashes=[
        <bytes32: 3aec608c1932a52fa8a753d094acca70180bef4973408b2e41362dd7d2626acc>
    ],
    finished_reward_slot_hashes=[
        <bytes32: b0e73e6b7657bf9681505be4777b015af9cd3da096a9131b389090090f26a885>
    ],
    sub_epoch_summary_included=None
)

In [8]:
await get_block_by_height_async(2)

FullBlock(
    finished_sub_slots=[
        EndOfSubSlotBundle(
            challenge_chain=ChallengeChainSubSlot(
                challenge_chain_end_of_slot_vdf=VDFInfo(
                    challenge=<bytes32: 0eb93be69bf0ea0dbb8033e7a78c8e0f03c742a08be363820a9d2b9562ab9acc>,
                    number_of_iterations=1024,
                    output=ClassgroupElement(
                        data=<bytes100: 00000800010100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>
                    )
                ),
                infused_challenge_chain_sub_slot_hash=None,
                subepoch_summary_hash=None,
                new_sub_slot_iters=None,
                new_difficulty=None
            ),
            infused_challenge_chain=InfusedChallengeChainSubSlot(
                infused_challenge_chain_end_of_slot_vdf=VDFInfo(
                    challenge=<bytes32: 144747c0afd162ab283331eaad4269afa7e76b6e9d198d28e35ae974010a6d34>,
                    number_of_iterations=1024,
                    output=ClassgroupElement(
                        data=<bytes100: 02000700010100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>
                    )
                )
            ),
            reward_chain=RewardChainSubSlot(
                end_of_slot_vdf=VDFInfo(
                    challenge=<bytes32: 6ceec1a03d2a512a79c9266205127ef2a2f7334f94c3bfe09b7d653d1bc507bf>,
                    number_of_iterations=890,
                    output=ClassgroupElement(
                        data=<bytes100: 00001100020100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>
                    )
                ),
                challenge_chain_sub_slot_hash=<bytes32: 83616d03da35d176e4c757f317e3c86a320adc3d495354e7c1330cfccc8cc8a9>,
                infused_challenge_chain_sub_slot_hash=<bytes32: 3aec608c1932a52fa8a753d094acca70180bef4973408b2e41362dd7d2626acc>,
                deficit=10
            ),
            proofs=SubSlotProofs(
                challenge_chain_slot_proof=VDFProof(
                    witness_type=0,
                    witness=b'\x03\x00\x17\x00\x02\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
                    normalized_to_identity=False
                ),
                infused_challenge_chain_slot_proof=VDFProof(
                    witness_type=0,
                    witness=b'\x00\x001\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
                    normalized_to_identity=False
                ),
                reward_chain_slot_proof=VDFProof(
                    witness_type=0,
                    witness=b'\x02\x00\x17\x00\x03\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [9]:
await get_coin_record_by_name_async('c5149017be6a5af5b9b876165137b3b04ba50ba079f92320a063e344e73c9130')

CoinRecord(
    coin=Coin { parent_coin_info: fe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000000, puzzle_hash: bd85c2c28bd770997c36037fa842fbbb719803dba7ff6b741148739576e68a39, amount: 2625000000000000000 },
    confirmed_block_index=1,
    spent_block_index=0,
    coinbase=True,
    timestamp=1668525007
)